In [21]:
import pandas as pd
import numpy as np

QuestionNumber = 5  # question per page

## Functions

In [22]:
def arrange_answers(answers):    # organize answers in one instance
    arranged_answers =[[] for _ in range(QuestionNumber)]
    comment = ""
    for name in answers.keys():
        if "less-2-checkbox" in name:
            continue
        answer = answers[name].strip().lower()    # remove sapce & convert to lower
        for i in range(QuestionNumber):
            if "answer"+str(i+1)+"-" in name:     # belong to question i
                arranged_answers[i].append(answer)
            if "in-answer-box"+str(i+1) in name:
                arranged_answers[i].append(answer)
            if name == "comments":
                comment = answer
    return [sorted(set(t), key=t.index) for t in  arranged_answers],comment # remove repeated answer # keep the answer order


In [25]:
# organize answers
batch_result = pd.read_csv("Batch_4282865_batch_results.csv")
hit_ques_stdans_video_time_answers_dict = dict()
comments = []

lasthitid = ""
for ri, (index, row) in enumerate(batch_result.iterrows()):
    chitid = row["HITId"]
    if chitid != lasthitid:
        hit_ques_stdans_video_time_answers_dict[chitid] = [dict() for _ in range(5)]
        returned_answers = row["Answer.taskAnswers"]
        canswers,comment = arrange_answers(eval(returned_answers)[0])
        if comment != "":
            comments.append(comment)
        for ind in range(5):
            hit_ques_stdans_video_time_answers_dict[chitid][ind]["question"] = row["Input.question"+str(ind+1)]
            hit_ques_stdans_video_time_answers_dict[chitid][ind]["video"] = "https://www.youtube.com/watch?v="+ row["Input.video"+str(ind+1)+"_id"]
            hit_ques_stdans_video_time_answers_dict[chitid][ind]["video_start_time"] = row["Input.video"+str(ind+1)+"_start_time"]
            hit_ques_stdans_video_time_answers_dict[chitid][ind]["video_end_time"] = row["Input.video"+str(ind+1)+"_end_time"]
            hit_ques_stdans_video_time_answers_dict[chitid][ind]["std_answer"] = row["Input.label"+str(ind+1)]
            hit_ques_stdans_video_time_answers_dict[chitid][ind]["answers"] = [canswers[ind]]
        lasthitid = chitid
    else:
        canswers,comment = arrange_answers(eval(row["Answer.taskAnswers"])[0])
        if comment != "":
            comments.append(comment)
        for ind in range(5):
            hit_ques_stdans_video_time_answers_dict[chitid][ind]["answers"].append(canswers[ind])
        

In [26]:
# write to file
with open("batch4(4282865)-val-1-20-results.txt","w") as f:
    for hit in hit_ques_stdans_video_time_answers_dict.values():
        for ques in hit:
            f.write("{}\n{}\nvideo time: {} {}\nstd_ans: {}\n\n"
                   .format(ques["question"],ques["video"],ques["video_start_time"],
                           ques["video_end_time"],ques["std_answer"]))
            for ans in ques["answers"]:
                f.write(str(ans)+"\n")
            f.write("\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n\n")
    f.write("Comments:\n")
    for comment in comments:
        f.write(comment)
        f.write("\n")